# ***Desafio 3***

### O Objetivo deste desafio é gerar um notebook que determine quão similares são os títulos do DATASET

In [2]:
import IPython
#install requirements if needed
from IPython.display import clear_output
!pip install Levenshtein
clear_output()

In [3]:
import pandas as pd
import numpy as np
import time
import csv
import os
import Levenshtein as lev

#### Para o cálculo de similaridade vou usar o pacote Levenshtein do Python.
#### A distância de Levenshtein é uma métrica de string para medir a diferença entre duas sequências. Normalmente, a distância Levenshtein entre duas palavras é o número mínimo de edições de um caractere (inserção, exclusão, substituição) necessárias para transformar uma palavra na outra.

Primeiramente vou ler o arquivo CSV com os dados

E faço uma pequena validação de nulos tal como verificar como são os dados em nossa base

In [4]:
nome_arquivo = 'items_titles_test.csv'
def ler_df(nome):
    df = pd.read_csv(nome) 
    print(df.info())
    print("Dataframe shape: ",df.shape)
    return df

In [5]:
data = ler_df(nome_arquivo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ITE_ITEM_TITLE  10000 non-null  object
dtypes: object(1)
memory usage: 78.2+ KB
None
Dataframe shape:  (10000, 1)


In [6]:
data.head()

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


# Solução desafio 3:

In [8]:
def similarity_sort_ram(data):
    start = time.time() 
    output_df = []
    #faremos um nó de iterações para varrer matricialmente a tabela item a item para comparar a similaridade
    for rowData1 in data.values:
        for rowData2 in data.values:
            if rowData1 == rowData2:
                #temos 2 opções aqui, caso estivermos interessados em manter a comparação entre os mesmos items retornamos 1 se não utilizaremos o break e ignoraremos esse registro
                simitarity = 1
                #break
            else:
                simitarity = lev.ratio(rowData1[0], rowData2[0])
            output = { "ITE_ITEM_TITLE_1": rowData1[0], "ITE_ITEM_TITLE_2": rowData2[0], 'Score Similitud (0,1)': simitarity} 
            output_df.append(output) # aqui encontramos a limitação desta solução que é manter em memória uma lista que cresce exponencialmente com o número de casos, veremos abaixo uma alternativa
    #aqui transformaremos a lista de outputs para um dataframe e faremos o sorting por maior score de similaridade
    output_df = pd.DataFrame(output_df).sort_values('Score Similitud (0,1)', ascending = False).reset_index(drop=True)
    
    end = time.time()
    T = end - start
    print("The time used to execute this program is: ", T/60, "minutes")
    
    return output_df

In [9]:
data_ordenada = similarity_sort_ram(data)
data_ordenada.head(15)

The time used to execute this program is:  13.673823010921478


,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,"Score Similitud (0,1)"
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids,1.0
1,Tênis Feminino P/ Usar C/ Vestido Edge Sumer E...,Tênis Feminino P/ Usar C/ Vestido Edge Sumer E...,1.0
2,Tênis Infantil Meia Minnie + Babuche + Mimo Ki...,Tênis Infantil Meia Minnie + Babuche + Mimo Ki...,1.0
3,Tênis Slip On Feminino Tweenie Pop 506007 Rosa...,Tênis Slip On Feminino Tweenie Pop 506007 Rosa...,1.0
4,Promoçã Sapatilha Mtb Northwave Rockster Preto...,Promoçã Sapatilha Mtb Northwave Rockster Preto...,1.0
5,Bike Absolute Feminina,Bike Absolute Feminina,1.0
6,Bicicleta Infantil Com Rodinhas - Aro 12 - Sel...,Bicicleta Infantil Com Rodinhas - Aro 12 - Sel...,1.0
7,Tênis Under Armour Bandit 6 Masculino Corrida ...,Tênis Under Armour Bandit 6 Masculino Corrida ...,1.0
8,Tenis Asics Gel Rocket 9 Mrh/pnk,Tenis Asics Gel Rocket 9 Mrh/pnk,1.0
9,Bicicleta Infantil Aro 12 Magic Rainbow Unicor...,Bicicleta Infantil Aro 12 Magic Rainbow Unicor...,1.0


Com o método acima não foi possível no meu sistema realizar a validação completa da base de "treinamento" com 30.000 casos, houve uma limitação em RAM, pois o processo guarda esses outputs em memória antes de transforma-lo no nosso DF resposta, percebemos aqui como se comporta o problema:

- 1.000 casos -> 1.000.000 resultados
- 10.000 casos -> 100.000.000 resultados
- 30.000 casos -> 900.000.000 resultados

Com isso em mente desenvolvi a solução abaixo, que não utiliza a memoria ram para alocar a lista de outputs e sim o disco, nesse processo podemos perder um pouco de performance em tempo de execução mas nos permite trabalhar com bases muito maiores sendo a limitação espaço em disco (mais barato e normalmente mais abundante em sistemas)

In [12]:
def similarity_sort_dump(data):
    start = time.time() 
    #diferentemente da solução acima, aqui nosso nó de iteração fica dentro de um nó de gravação em disco para criar nosso arquivo csv com intenção de liberar memória RAM
    with open('dump.csv','w',encoding="utf-8") as dataDump:
        writer=csv.writer(dataDump)
        for rowData1 in data.values:
            for rowData2 in data.values:
                if rowData1 == rowData2:
                    #temos 2 opções aqui, caso estivermos interessados em manter a comparação entre os mesmos items retornamos 1 se não utilizaremos o break e ignoraremos esse registro
                    simitarity = 1
                    #break
                else:
                    simitarity = lev.ratio(rowData1[0], rowData2[0])
                output = [rowData1[0],rowData2[0],simitarity]
                writer.writerow(output)
    #aqui vamos ler nosso arquivo csv gerado por nossa iteração e dump, criaremos o dataframe e organizamos pelo maior score de similaridade
    output_df = pd.read_csv('dump.csv', header =0, names=['ITE_ITEM_TITLE_1','ITE_ITEM_TITLE_2','Score Similitud (0,1)']).sort_values('Score Similitud (0,1)', ascending = False).reset_index(drop=True)
    
    end = time.time()
    T = end - start
    print("The time used to execute this program is: ", T/60, "minutes")
    #antes de finalizar o processo limpamos o arquivo gerado
    os.remove("dump.csv")
    return output_df

In [11]:
data_ordenada = similarity_sort_dump(data)
data_ordenada.head(15)

The time used to execute this program is:  13.071378739674886


,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,"Score Similitud (0,1)"
0,Tênis Polo Ralph Lauren Modelo Cantor Low Bran...,Tênis Polo Ralph Lauren Modelo Cantor Low Bran...,1.0
1,Tênis Beira Rio Conforto Flat Feminino,Tênis Beira Rio Conforto Flat Feminino,1.0
2,Tênis Infantil Molekinho Esportivo Leve E Conf...,Tênis Infantil Molekinho Esportivo Leve E Conf...,1.0
3,Tênis Velcro Plataforma Casual Kolosh C2283 Preto,Tênis Velcro Plataforma Casual Kolosh C2283 Preto,1.0
4,Tênis Slip On Infantil Baby Personagens,Tênis Slip On Infantil Baby Personagens,1.0
5,Tênis Casual Com Cadarço Moleca 5667-302,Tênis Casual Com Cadarço Moleca 5667-302,1.0
6,Tenis Brinde Bolsa Infantil Feminino Menina Ca...,Tenis Brinde Bolsa Infantil Feminino Menina Ca...,1.0
7,Tênis Feminino Via Marte Branco - 2111401-04,Tênis Feminino Via Marte Branco - 2111401-04,1.0
8,Tênis Infantil Bibi Luz Led Space Wave Masculi...,Tênis Infantil Bibi Luz Led Space Wave Masculi...,1.0
9,Tênis Feminino Sport Fino Casual Jeans Azul De...,Tênis Feminino Sport Fino Casual Jeans Azul De...,1.0
